# ECLIPSE

In [ ]:
#importing necessary libraries 
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
import requests
import json
import urllib.request
import time
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
import re
from sklearn.metrics.pairwise import cosine_similarity
import statistics
from datetime import datetime
from scipy.stats import skew

In [ ]:
e_data = pd.read_csv("eclipse_final_preprocessed.csv",engine='python')
e_data

In [ ]:
dup = e_data[e_data['Duplicate_Bug_Ids'] > 0]
dup

In [ ]:
dup_list = list(dup.Duplicate_Bug_Ids)

In [ ]:
dup_list[:10]

In [ ]:
bug_list = list(dup["Bug_ID"])

In [ ]:
bug_list[:10]

In [ ]:
#calculating how many duplicate ids are present as the bug ids in the whole dataset in last five years
count_id = 0
dup_id_in_bug_id = []
for i,v in enumerate(dup_list):
    if (v in bug_list):
        count_id +=1
        dup_id_in_bug_id.append(v)

In [ ]:
len(dup_id_in_bug_id)

In [ ]:
dup_id_not_in_bug_id  = [x for x in dup_list if x not in dup_id_in_bug_id ] #length of duplicate id which is not present in bug ids
len(dup_id_not_in_bug_id)

In [ ]:
dup_id_not_in_bug_id2 = []
for i,v in enumerate(dup_id_not_in_bug_id):
    v = int(v)
    dup_id_not_in_bug_id2.append(v)
dup_id_not_in_bug_id2[:10]

In [ ]:
#Fetching description
resp = None
failed_count_dup = 0
duplicates_dup = []
comments_dup = {}
tick = datetime.now()
total = len(dup_id_not_in_bug_id2)
for idx, val in enumerate(dup_id_not_in_bug_id2):
    bug_url = 'https://bugs.eclipse.org/bugs/rest/bug/'+str(val)+'/comment'
    resp = requests.get(bug_url)
    try:
        desc_dup = list(filter(lambda comments_dup: comments_dup['count'] == 0, resp.json()['bugs'][str(val)]['comments']))[0]['text']
        comments_dup[val] = desc_dup
        tock = datetime.now()
        delta = tock - tick
        with urllib.request.urlopen("https://bugs.eclipse.org/bugs/rest/bug/"+str(val)) as url:
            bug_data = json.loads(url.read().decode())
        duplicates_dup.append(bug_data['bugs'][0]['dupe_of'])
        print(f'\r{idx}, Elapsed: {delta}, Remaining: {delta/idx * total}', end='')
    except Exception as e:
        failed_count_dup += 1

In [ ]:
failed_count_dup

In [ ]:
len(duplicates_dup)

In [ ]:
dup_desc = pd.DataFrame(comments_dup.items(), columns=['Dup Id', 'Description'])
# dup_desc['Dup_Bug_Ids'] = duplicates_dup
dup_desc

In [ ]:
resp = None
failed_count_dup = 0
duplicates_dup = []

comments_dup = {}
summary_dup = {}
creation_dup = {}
resolution_dup = {}
status_dup = {}
product_dup = {}
component_dup = {}

tick = datetime.now()
total = len(dup_id_not_in_bug_id)

In [ ]:
#Fetching all other features
for idx, val in enumerate(dup_id_not_in_bug_id2):
    
    bug_url = 'https://bugs.eclipse.org/bugs/rest/bug/'+str(val)
    resp = requests.get(bug_url)
    
    try:
        summ_dup = resp.json()['bugs'][0]['summary']
        summary_dup[val] = summ_dup
        create_dup =  resp.json()['bugs'][0]['creation_time']
        creation_dup[val] = create_dup
        
        resolve_dup = resp.json()['bugs'][0]['resolution']
        resolution_dup[val] = resolve_dup
        
        stat_dup = resp.json()['bugs'][0]['status']
        status_dup[val] = stat_dup
        
        prod_dup = resp.json()['bugs'][0]['product']
        product_dup[val] = prod_dup
        
        comp_dup = resp.json()['bugs'][0]['component']
        component_dup[val] = comp_dup
        
        tock = datetime.now()
        delta = tock - tick
        with urllib.request.urlopen("https://bugs.eclipse.org/bugs/rest/bug/"+str(val)) as url:
            bug_data = json.loads(url.read().decode())
        duplicates_dup.append(bug_data['bugs'][0]['dupe_of'])
        print(f'\r{idx}, Elapsed: {delta}, Remaining: {delta/idx * total}', end='')
    except Exception as e:
        failed_count_dup += 1

In [ ]:
dup_summary = pd.DataFrame(summary_dup.items(), columns=['Dup Id', 'Summary'])
# dup_summary['Dup_Bug_Ids'] = duplicates_dup
dup_summary

In [ ]:
dup_createdtime = pd.DataFrame(creation_dup.items(), columns=['Dup Id', 'Created_time'])
# dup_createdtime['Dup_Bug_Ids'] = duplicates_dup
dup_createdtime

In [ ]:
dup_resolution = pd.DataFrame(resolution_dup.items(), columns=['Dup Id', 'Resolution'])
# dup_resolution['Dup_Bug_Ids'] = duplicates_dup
dup_resolution

In [ ]:
dup_status = pd.DataFrame(status_dup.items(), columns=['Dup Id', 'Status'])
# dup_status['Dup_Bug_Ids'] = duplicates_dup
dup_status

In [ ]:
dup_product = pd.DataFrame(product_dup.items(), columns=['Dup Id', 'Product'])
# dup_product['Dup_Bug_Ids'] = duplicates_dup
dup_product

In [ ]:
dup_component = pd.DataFrame(component_dup.items(), columns=['Dup Id', 'Component'])
# dup_component['Dup_Bug_Ids'] = duplicates_dup
dup_component

In [ ]:
data_merged = pd.merge(dup_desc,dup_summary,left_on="Dup Id",right_on= "Dup Id")
data_merged

In [ ]:
data_merged = pd.merge(data_merged,dup_createdtime,left_on="Dup Id",right_on= "Dup Id")
data_merged

In [ ]:
data_merged = pd.merge(data_merged,dup_resolution,left_on="Dup Id",right_on= "Dup Id")
data_merged

In [ ]:
data_merged = pd.merge(data_merged,dup_status,left_on="Dup Id",right_on= "Dup Id")
data_merged

In [ ]:
data_merged = pd.merge(data_merged,dup_product,left_on="Dup Id",right_on= "Dup Id")
data_merged

In [ ]:
data_merged = pd.merge(data_merged,dup_component,left_on="Dup Id",right_on= "Dup Id")
data_merged

In [ ]:
fetched_duplicate_data_eclipse= data_merged[['Dup Id','Product','Component','Status','Resolution','Summary','Created_time','Description']]
fetched_duplicate_data_eclipse

In [ ]:
bug_list = list(fetched_duplicate_data_eclipse["Dup Id"])

In [ ]:
#calculating how many duplicate ids are present as the bug ids in the whole dataset in last five years
count_id = 0
dup_id_in_bug_id = []
for i,v in enumerate(dup_list):
    if (v in bug_list):
        count_id +=1
        dup_id_in_bug_id.append(v)

In [ ]:
len(dup_id_in_bug_id)

In [ ]:
duplicates_dup = list(dict.fromkeys(duplicates_dup))
len(duplicates_dup)

In [ ]:
import math

In [ ]:
fetched_duplicate_data_eclipse['Dup_Bug_Ids'] =  math.nan
fetched_duplicate_data_eclipse

In [ ]:
fetched_duplicate_data_eclipse =  fetched_duplicate_data_eclipse.rename(columns={"Bug_Id": "Bug_ID", "Created_time": "Changed"})
fetched_duplicate_data_eclipse

In [ ]:
fetched_duplicate_data_eclipse.to_csv("fetched_eclipse.csv",index=False)

In [ ]:
e_data

In [ ]:
fetched_duplicate_data_eclipse

In [ ]:
final_eclipse_data = pd.concat([e_data,fetched_duplicate_data_eclipse])
final_eclipse_data

In [ ]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

for index, row in final_eclipse_data.iterrows():
    des =  str(row.Description)
    des = des.replace("\n"," ")
    des = cleanhtml(des)
    if(len(des)>=500):
        row.Description = des[:500]
        
#Text Cleaning round 1 (removing punctutions)
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     print(text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('\w*\f\w*', ' ', text)
    text = re.sub('\(.*?\)', ' ', text)
    text = re.sub('\[.*]\)', ' ', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+',' ',text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

final_eclipse_data = final_eclipse_data.dropna(axis=0, subset=['Description'])
final_eclipse_data['Description'] = final_eclipse_data['Description'].apply(clean_text_round1)

# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

final_eclipse_data['Description'] = final_eclipse_data['Description'].apply(clean_text_round2)

In [ ]:
final_eclipse_data = final_eclipse_data.reset_index(drop=True)

In [ ]:
final_eclipse_data

In [ ]:
final_eclipse_data.to_csv("final_eclipse_data.csv",index=False)

In [ ]:
final_eclipse_data

In [ ]:
dup = final_eclipse_data[final_eclipse_data['Duplicate_Bug_Ids'] > 0]
dup

In [ ]:
dup_list = list(dup.Duplicate_Bug_Ids)
dup_list[:10]

In [ ]:
bug_list = list(final_eclipse_data["Bug_ID"])

In [ ]:
#calculating how many duplicate ids are present as the bug ids in the whole dataset in last five years
count_id = 0
dup_id_in_bug_id = []
for i,v in enumerate(dup_list):
    if (v in bug_list):
        count_id +=1
        dup_id_in_bug_id.append(v)

In [ ]:
len(dup_id_in_bug_id)

In [ ]:
dup_id_not_in_bug_id  = [x for x in dup_list if x not in dup_id_in_bug_id ] #length of duplicate id which is not present in bug ids
len(dup_id_not_in_bug_id)

In [ ]:
dup_id_not_in_bug_id

In [ ]:
final_eclipse_data = final_eclipse_data[~final_eclipse_data.Duplicate_Bug_Ids.isin(dup_id_not_in_bug_id)]

In [ ]:
final_eclipse_data

In [ ]:
final_eclipse_data.to_csv("final_eclipse_data.csv",index=False)

In [ ]:
final_eclipse_data = final_eclipse_data.reset_index(drop=True)
final_eclipse_data

# final data

In [ ]:
nan_value = float("NaN")
final_eclipse_data.replace("", nan_value, inplace=True)
final_eclipse_data.dropna(subset = ["Description"], inplace=True)

In [ ]:
final_eclipse_data.info()

In [ ]:
dup = final_eclipse_data[final_eclipse_data['Duplicate_Bug_Ids'] > 0]
dup

In [ ]:
dup.to_csv("final_eclipse_duplicate.csv",index=False)

# duplicates

In [ ]:
dup.info()

# Train, Test

In [ ]:
import random
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
training_dataset ,test_dataset= sklearn.model_selection.train_test_split(final_eclipse_data, train_size=0.8, test_size=0.2)

In [ ]:
len(training_dataset)

In [ ]:
len(test_dataset)

In [ ]:
nan_value = float("NaN")
training_dataset.replace("", nan_value, inplace=True)
training_dataset.dropna(subset = ["Description"], inplace=True)

In [ ]:
cond = (training_dataset['Description'] == 'NaN')
cond.unique()

In [ ]:
nan_value = float("NaN")
test_dataset.replace("", nan_value, inplace=True)
test_dataset.dropna(subset = ["Description"], inplace=True)

In [ ]:
cond = (test_dataset['Description'] == 'NaN')
cond.unique()

In [ ]:
training_dataset.to_csv("eclipse_training_dataset_final.csv",index = None)

In [ ]:
test_dataset.to_csv("eclipse_testing_dataset_final.csv",index = None)

In [ ]:
duplicate_test = test_dataset[test_dataset['Duplicate_Bug_Ids'] > 0]
duplicate_test

# Duplicate data splitting to textually similar and dissimilar

In [ ]:
ted = dup['Description']
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Generate matrix of word vectors
tfidf_matrix = vectorizer.fit_transform(ted)

# Print the shape of tfidf_matrix
print(tfidf_matrix.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the tf-idf vectors for the corpus
tfidf_matrix = tfidf_vectorizer.fit_transform(ted)

# compute and print the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

In [ ]:
print(len(cosine_sim))
print(cosine_sim.shape)

In [ ]:
import numpy as np

In [ ]:
avg =np.average(cosine_sim, axis=1)
cosine_sim[1]

In [ ]:
avg_df = pd.DataFrame(avg, columns = ['Avg_Sim_Score'])

print(avg_df)
print(type(avg_df))

In [ ]:
avg_for_sorting = pd.DataFrame(avg_df)

In [ ]:
sorted= avg_for_sorting.sort_values(by='Avg_Sim_Score', ascending=False)

In [ ]:
from scipy.stats import skew
#testing skewness of distribution
print(skew(avg))
print(skew(avg, bias=False))

In [ ]:
import statistics
# Calculate median values
print(statistics.median(avg))

In [ ]:
dup.insert(loc=7,
          column='Average_Cos_Similarity_Score',
          value=avg_df.values)

In [ ]:
score_similar = dup[dup['Average_Cos_Similarity_Score'] >= 0.045767579138220424]
score_similar

In [ ]:
score_dissimilar = dup[dup['Average_Cos_Similarity_Score'] < 0.045767579138220424]
score_dissimilar

In [ ]:
score_similar.info()

In [ ]:
score_dissimilar.info()

In [ ]:
score_similar.to_csv("eclipse_textually_similar_duplicates.csv",index=False)

In [ ]:
score_dissimilar.to_csv("eclipse_textually_dissimilar_duplicates.csv",index=False)

# Splitting Testing data

In [ ]:
ted = duplicate_test['Description']
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Generate matrix of word vectors
tfidf_matrix = vectorizer.fit_transform(ted)

# Print the shape of tfidf_matrix
print(tfidf_matrix.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the tf-idf vectors for the corpus
tfidf_matrix = tfidf_vectorizer.fit_transform(ted)

# compute and print the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

In [ ]:
print(len(cosine_sim))
print(cosine_sim.shape)

In [ ]:
import numpy as np

In [ ]:
avg =np.average(cosine_sim, axis=1)
cosine_sim[1]

In [ ]:
avg_df = pd.DataFrame(avg, columns = ['Avg_Sim_Score'])

print(avg_df)
print(type(avg_df))

In [ ]:
avg_for_sorting = pd.DataFrame(avg_df)

In [ ]:
sorted= avg_for_sorting.sort_values(by='Avg_Sim_Score', ascending=False)

In [ ]:
from scipy.stats import skew
#testing skewness of distribution
print(skew(avg))
print(skew(avg, bias=False))

In [ ]:
import statistics
# Calculate median values
print(statistics.median(avg))

In [ ]:
duplicate_test.insert(loc=7,
          column='Average_Cos_Similarity_Score',
          value=avg_df.values)

In [ ]:
score_similar = duplicate_test[duplicate_test['Average_Cos_Similarity_Score'] >= 0.050248503921723954]
score_similar

In [ ]:
score_dissimilar = duplicate_test[duplicate_test['Average_Cos_Similarity_Score'] < 0.050248503921723954]
score_dissimilar

In [ ]:
score_similar.to_csv("eclipse_textually_similar_testset_duplicates.csv",index=False)

In [ ]:
score_dissimilar.to_csv("eclipse_textually_dissimilar_testset_duplicates.csv",index=False)